In [ ]:
import pandas as pd
import pvlib
from pvlib import location

# Define date range for GMT+7 (Asia/Saigon)
start_date = pd.Timestamp("2025-01-01 00:00:00", tz="Asia/Saigon")
end_date = pd.Timestamp("2025-03-14 23:59:59", tz="Asia/Saigon")

# Using existing site location
lat, lon = 10.76477848, 106.3148294
site = location.Location(lat, lon, tz="Asia/Saigon")

# Generate time range with 10-minute intervals
times = pd.date_range(start=start_date, end=end_date, freq="10min", tz="Asia/Saigon")

# Calculate clear sky data
clearsky_data = site.get_clearsky(times)
clearsky_data.reset_index(inplace=True)
clearsky_data.rename(columns={"index": "Time"}, inplace=True)
clearsky_data.Time = pd.to_datetime(clearsky_data.Time).dt.tz_localize(None)

# Index là cột Time, tạo một 'GHI_CS' mới từ cột 'ghi' của clearsky_data và xoá các cột còn lại
clearsky_data.set_index("Time", inplace=True)
clearsky_data = clearsky_data[["ghi"]]
clearsky_data.rename(columns={"ghi": "GHI_CS"}, inplace=True)

In [55]:
clearsky_data

In [ ]:
# %% Cell 1: Import thư viện và cài đặt
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.path.append("P:/7. User/Tuong")

from DB_ALL import SQL_NLTT_NOIBO, Database_TTD

PATH = r"Z:\Sky-image\namnvn\Data_process\MT Solarpark 1"
POWER_CSV = r"C:\Khue\H9_Solar_Power_Forecasting\data\processed\df_353.csv"
SEQ_LENGTH = 24
PRED_LENGTH = 24
IMG_SIZE = (32, 32)
NUM_CHANNELS = 4
BATCH_SIZE = 128
EPOCHS = 30

START_DATE = datetime.datetime(2025, 1, 1)
END_DATE = datetime.datetime(2025, 1, 14)

print("✅ Đã import thư viện và thiết lập cấu hình")

In [ ]:
# %% Cell 2: Xử lý dữ liệu công suất
print("🔄 Đang load dữ liệu công suất...")
power_df = pd.read_csv(POWER_CSV)
power_df.reset_index(inplace=True)
power_df["Time"] = pd.to_datetime(power_df["Time"])
scaler = MinMaxScaler()
power_df["P_uoc_normalized"] = scaler.fit_transform(power_df[["P_uoc"]])
power_lookup = power_df.set_index("Time")["P_uoc_normalized"].to_dict()
print(f"✅ Đã load {len(power_df)} bản ghi")

print("Dữ liệu mẫu:", power_df.head())
print("Lookup mẫu:", {k: power_lookup[k] for k in list(power_lookup.keys())[:5]})

In [ ]:
# %% Cell 3.1: Xử lý ảnh thành chuỗi dữ liệu multi-channel
def load_and_process_image(path):
    try:
        img = tf.io.read_file(path)
        img = tf.image.decode_image(img, channels=1, expand_animations=False)
        img = tf.image.resize(img, IMG_SIZE)
        img = (img / 127.5) - 1.0
        return img.numpy()
    except:
        return np.zeros(IMG_SIZE + (1,), dtype=np.float32)


def preprocess_images(image_records_dict):
    print("\n🔄 Đang xử lý ảnh thành chuỗi dữ liệu multi-channel...")
    prefixes = ["b03_", "b07_", "b08_", "b13_"]
    sequences_images = []
    sequences_times = []

    # Tìm thời gian chung
    times = set.intersection(
        *(set(dt for _, dt in image_records_dict[prefix]) for prefix in prefixes)
    )
    times = sorted(times)
    min_length = len(times) - SEQ_LENGTH - PRED_LENGTH + 1

    for i in range(max(0, min_length)):
        time_window = times[i : i + SEQ_LENGTH]
        if len(time_window) != SEQ_LENGTH:  # Bỏ qua nếu không đủ SEQ_LENGTH
            continue
        multi_channel_seq = []
        for prefix in prefixes:
            channel_seq = []
            for t in time_window:
                path = next(
                    (p for p, dt in image_records_dict[prefix] if dt == t), None
                )
                img = (
                    load_and_process_image(path)
                    if path
                    else np.zeros(IMG_SIZE + (1,), dtype=np.float32)
                )
                channel_seq.append(img.squeeze())  # Loại bỏ chiều (1)
            multi_channel_seq.append(channel_seq)
        # Stack thành (SEQ_LENGTH, 32, 32, 4)
        stacked_seq = np.stack(multi_channel_seq, axis=-1)
        sequences_images.append(stacked_seq)
        sequences_times.append(time_window[-1])

    sequences_images = np.array(sequences_images)
    print(f"✅ Đã xử lý {len(sequences_images)} chuỗi ảnh")
    print("Shape của sequences_images:", sequences_images.shape)
    if sequences_images.size > 0:
        print(
            "Mẫu đầu tiên (min, max):",
            sequences_images[0].min(),
            sequences_images[0].max(),
        )
    return sequences_images, sequences_times


# Tạo image_records
date_dirs = [d.strftime("%Y%m%d") for d in pd.date_range(START_DATE, END_DATE)]
image_records_dict = {"b03_": [], "b07_": [], "b08_": [], "b13_": []}
for date_dir in date_dirs:
    date_path = os.path.join(PATH, date_dir)
    if os.path.exists(date_path):
        for time_dir in os.listdir(date_path):
            time_path = os.path.join(date_path, time_dir)
            if os.path.isdir(time_path):
                try:
                    dt = datetime.datetime.strptime(
                        f"{date_dir}T{time_dir}", "%Y%m%dT%H%M"
                    )
                    for file in os.listdir(time_path):
                        for prefix in image_records_dict.keys():
                            if file.lower().startswith(
                                prefix
                            ) and file.lower().endswith(
                                (".jpg", ".jpeg", ".png", ".gif", ".bmp")
                            ):
                                img_path = os.path.join(time_path, file)
                                image_records_dict[prefix].append((img_path, dt))
                except:
                    continue

for prefix in image_records_dict:
    image_records_dict[prefix].sort(key=lambda x: x[1])

sequences_images_processed, sequences_times = preprocess_images(image_records_dict)

# Kiểm tra
if sequences_images_processed.size > 0:
    for i, prefix in enumerate(["b03_", "b07_", "b08_", "b13_"]):
        plt.subplot(1, 4, i + 1)
        plt.imshow(sequences_images_processed[0][0, :, :, i], cmap="gray")
        plt.title(f"Kênh {prefix}")
    plt.tight_layout()
    plt.show()
    print("Thời gian mẫu:", [t.strftime("%Y-%m-%d %H:%M") for t in sequences_times[:5]])
else:
    print("⚠️ Không có chuỗi ảnh nào được tạo!")


In [ ]:
# %% Cell 3.2: Ghép với P_uoc và tạo dataset
def prepare_dataset_with_processed_images(sequences_images_processed, sequences_times):
    print("\n🔄 Đang ghép dữ liệu ảnh với P_uoc...")
    sequences_p_uoc = []
    targets = []
    valid_sequences_images = []
    valid_sequences_times = []
    for i, (img_seq, seq_time) in enumerate(
        zip(sequences_images_processed, sequences_times)
    ):
        start_idx = i
        p_uoc_seq = [
            power_lookup.get(image_records_dict["b03_"][start_idx + j][1], 0.0)
            for j in range(SEQ_LENGTH)
        ]
        target_dts = [
            image_records_dict["b03_"][start_idx + SEQ_LENGTH + j][1]
            for j in range(PRED_LENGTH)
        ]
        target_powers = [power_lookup.get(dt, -1.0) for dt in target_dts]
        if all(power != -1.0 for power in target_powers):
            valid_sequences_images.append(img_seq)
            sequences_p_uoc.append(p_uoc_seq)
            targets.append(target_powers)
            valid_sequences_times.append(seq_time)

    print("\nVí dụ đầu vào và đầu ra:")
    print("P_uoc đầu vào:", sequences_p_uoc[0])
    print("P_uoc đầu ra:", targets[0])

    ds = tf.data.Dataset.from_tensor_slices(
        (
            (np.array(valid_sequences_images), np.array(sequences_p_uoc)),
            np.array(targets),
        )
    )
    ds = ds.map(lambda x, y: ((x[0], x[1]), y), num_parallel_calls=tf.data.AUTOTUNE)
    final_ds = ds.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    print(f"✅ Dataset đã sẵn sàng với {len(valid_sequences_images)} mẫu")
    return final_ds, valid_sequences_times


dataset, sequences_times_train = prepare_dataset_with_processed_images(
    sequences_images_processed, sequences_times
)

In [ ]:
# %% Cell 4: Định nghĩa mô hình
def build_model():
    input_images = layers.Input(
        shape=(SEQ_LENGTH, IMG_SIZE[0], IMG_SIZE[1], NUM_CHANNELS)
    )
    x = layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation="relu"))(
        input_images
    )
    x = layers.TimeDistributed(layers.MaxPooling2D(2, 2))(x)
    x = layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation="relu"))(x)
    x = layers.TimeDistributed(layers.GlobalAveragePooling2D())(x)
    input_p_uoc = layers.Input(shape=(SEQ_LENGTH,))
    p_uoc_features = layers.Reshape((SEQ_LENGTH, 1))(input_p_uoc)
    combined = layers.Concatenate(axis=-1)([x, p_uoc_features])
    lstm_out = layers.LSTM(128, return_sequences=False)(combined)
    output = layers.Dense(PRED_LENGTH)(lstm_out)
    model = models.Model(inputs=[input_images, input_p_uoc], outputs=output)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001), loss="mae", metrics=["mae"]
    )
    print("✅ Mô hình đã được build")
    model.summary()
    return model


model = build_model()

In [ ]:
# %% Cell 5: Huấn luyện mô hình
def train_model(dataset):
    model = build_model()
    print("\n🔄 Đang huấn luyện mô hình...")
    history = model.fit(dataset, epochs=EPOCHS, verbose=1)
    print("✅ Huấn luyện hoàn tất")
    return model, history


model, history = train_model(dataset)
plt.plot(history.history["loss"], label="Loss")
plt.plot(history.history["mae"], label="MAE")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.title("Training Loss and MAE")
plt.legend()
plt.show()

In [ ]:
def test_forecast_interval(model, scaler, test_date, input_start_hour, input_end_hour):
    """
    Test dự báo công suất cho một khoảng thời gian cụ thể trên một ngày.

    Đầu vào:
      - model: mô hình đã được huấn luyện.
      - scaler: đối tượng MinMaxScaler đã dùng để chuẩn hóa công suất.
      - test_date: ngày test (ví dụ: datetime.datetime(2025, 1, 18)).
      - input_start_hour: giờ bắt đầu đầu vào (ví dụ: 10).
      - input_end_hour: giờ kết thúc đầu vào (ví dụ: 14).

    Giả sử thời gian đầu vào và thời gian forecast có độ dài bằng nhau.
    Với mỗi bước cách nhau 10 phút, nên:
      SEQ_LENGTH = số bước đầu vào = (input_end_hour - input_start_hour)*6
      PRED_LENGTH = số bước forecast = (input_end_hour - input_start_hour)*6

    Trả về:
      - DataFrame có chỉ số thời gian, chứa giá trị dự báo và giá trị thực (nếu có) cho khoảng forecast.
    """
    import os
    import datetime
    import numpy as np
    import pandas as pd

    # Xác định khoảng thời gian đầu vào và forecast
    input_duration = input_end_hour - input_start_hour  # ví dụ: 14 - 10 = 4 giờ
    forecast_end_hour = input_end_hour + input_duration  # ví dụ: 14 + 4 = 18 giờ

    # Xây dựng các mốc thời gian
    test_date_only = test_date.date()
    input_start_dt = datetime.datetime.combine(
        test_date_only, datetime.time(input_start_hour, 0)
    )
    forecast_end_dt = datetime.datetime.combine(
        test_date_only, datetime.time(forecast_end_hour, 0)
    )
    total_steps = int(
        ((forecast_end_dt - input_start_dt).seconds) / 600
    )  # mỗi bước 10 phút

    # Kiểm tra tổng số bước có bằng tổng SEQ_LENGTH + PRED_LENGTH không
    if total_steps != (SEQ_LENGTH + PRED_LENGTH):
        print(
            f"Warning: Tổng số bước ({total_steps}) khác với SEQ_LENGTH+PRED_LENGTH ({SEQ_LENGTH + PRED_LENGTH})."
        )

    # Danh sách các timestamp cho toàn bộ khoảng (từ 10h đến 18h)
    full_timestamps = [
        input_start_dt + datetime.timedelta(minutes=10 * i) for i in range(total_steps)
    ]

    # --- Load dữ liệu ảnh của ngày test trong khoảng từ input_start_dt đến forecast_end_dt ---
    test_image_records_dict = {"b03_": [], "b07_": [], "b08_": [], "b13_": []}
    date_dir = test_date.strftime("%Y%m%d")
    date_path = os.path.join(PATH, date_dir)
    if not os.path.exists(date_path):
        print(f"Không tìm thấy thư mục dữ liệu cho ngày {date_dir}.")
        return None

    for time_dir in os.listdir(date_path):
        time_path = os.path.join(date_path, time_dir)
        if os.path.isdir(time_path):
            try:
                dt = datetime.datetime.strptime(f"{date_dir}T{time_dir}", "%Y%m%dT%H%M")
                # Chỉ chọn các timestamp trong khoảng từ input_start_dt đến forecast_end_dt
                if input_start_dt <= dt < forecast_end_dt:
                    for file in os.listdir(time_path):
                        for prefix in test_image_records_dict.keys():
                            if file.lower().startswith(
                                prefix
                            ) and file.lower().endswith(
                                (".jpg", ".jpeg", ".png", ".gif", ".bmp")
                            ):
                                img_path = os.path.join(time_path, file)
                                test_image_records_dict[prefix].append((img_path, dt))
            except Exception as e:
                print(f"Lỗi xử lý thư mục {time_path}: {e}")
                continue
    # Sắp xếp theo thời gian cho mỗi kênh
    for prefix in test_image_records_dict:
        test_image_records_dict[prefix].sort(key=lambda x: x[1])

    # --- Xây dựng chuỗi ảnh theo toàn bộ khoảng (10h đến 18h) ---
    # Vì hàm preprocess_images ban đầu chỉ tạo ra các chuỗi có độ dài SEQ_LENGTH,
    # ta tự tạo chuỗi đầy đủ dựa vào full_timestamps.
    prefixes = ["b03_", "b07_", "b08_", "b13_"]
    # Tạo dictionary cho từng kênh: mapping timestamp -> image (nếu có)
    image_dict = {prefix: {} for prefix in prefixes}
    for prefix in prefixes:
        for path, ts in test_image_records_dict[prefix]:
            image_dict[prefix][ts] = load_and_process_image(path)

    full_seq = []
    for ts in full_timestamps:
        multi_channel_imgs = []
        for prefix in prefixes:
            # Nếu không có ảnh tại thời điểm ts, thay bằng mảng zeros
            img = image_dict[prefix].get(
                ts, np.zeros(IMG_SIZE + (1,), dtype=np.float32)
            )
            multi_channel_imgs.append(img.squeeze())
        # Ghép các kênh lại thành mảng với shape (IMG_SIZE[0], IMG_SIZE[1], NUM_CHANNELS)
        img_stack = np.stack(multi_channel_imgs, axis=-1)
        full_seq.append(img_stack)
    full_seq = np.array(full_seq)  # shape: (total_steps, 32, 32, 4)

    # Tách dữ liệu thành phần đầu vào và phần cần dự báo
    input_images = full_seq[:SEQ_LENGTH]  # từ 10h đến 14h
    forecast_images = full_seq[
        SEQ_LENGTH : SEQ_LENGTH + PRED_LENGTH
    ]  # từ 14h đến 18h (dùng để lấy timestamp & giá trị thực nếu có)

    # Lấy chuỗi công suất tương ứng từ power_lookup
    input_power_seq = [power_lookup.get(ts, 0.0) for ts in full_timestamps[:SEQ_LENGTH]]
    # Get true power values for forecast period and inverse transform them
    true_forecast_power = [
        power_lookup.get(ts, np.nan)
        for ts in full_timestamps[SEQ_LENGTH : SEQ_LENGTH + PRED_LENGTH]
    ]
    true_forecast_power = np.array(true_forecast_power).reshape(-1, 1)
    true_forecast_power = scaler.inverse_transform(true_forecast_power).flatten()

    # Chuẩn bị dữ liệu cho mô hình (thêm chiều batch)
    x_image = np.expand_dims(input_images, axis=0)  # (1, SEQ_LENGTH, 32, 32, 4)
    x_power = np.expand_dims(np.array(input_power_seq), axis=0)  # (1, SEQ_LENGTH)

    # Dự báo với mô hình
    y_pred = model.predict([x_image, x_power])
    # y_pred có shape (1, PRED_LENGTH)
    y_pred = y_pred[0]  # shape: (PRED_LENGTH,)
    # Đưa về thang đo ban đầu
    y_pred_original = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

    # Tạo DataFrame với chỉ số thời gian cho phần forecast (14h - 18h)
    forecast_timestamps = full_timestamps[SEQ_LENGTH : SEQ_LENGTH + PRED_LENGTH]
    df_forecast = pd.DataFrame(
        {
            "Timestamp": forecast_timestamps,
            "Predicted_Power": y_pred_original,
            "True_Power": true_forecast_power,
        }
    )
    df_forecast.set_index("Timestamp", inplace=True)

    return df_forecast


# --- Test Case ---
import datetime
import matplotlib.pyplot as plt

# Test cho ngày 18/01/2025, đầu vào từ 10h đến 14h, dự báo từ 14h đến 18h
test_date = datetime.datetime(2025, 1, 18)
input_start_hour = 10
input_end_hour = 14

df_result = test_forecast_interval(
    model, scaler, test_date, input_start_hour, input_end_hour
)


In [ ]:
def test_multiple_intervals(model, scaler, test_date, num_intervals=3):
    """
    Thực hiện dự báo nhiều khoảng thời gian liên tiếp, mỗi khoảng 4 giờ

    Args:
        model: Mô hình đã train
        scaler: MinMaxScaler object
        test_date: Ngày test
        num_intervals: Số khoảng dự báo (mặc định là 3: 10-14h → 14-18h → 18-22h → 22-02h)

    Returns:
        List các DataFrame kết quả dự báo
    """
    results = []

    for i in range(num_intervals):
        # Tính giờ bắt đầu và kết thúc cho mỗi khoảng
        input_start = 10 + (i * 4)  # 10, 14, 18
        input_end = input_start + 4  # 14, 18, 22

        # Xử lý trường hợp chuyển ngày (sau 24h)
        current_date = test_date
        if input_start >= 24:
            input_start -= 24
            input_end -= 24
            current_date += datetime.timedelta(days=1)

        print(f"\nDự báo khoảng {i + 1}:")
        print(f"Đầu vào: {input_start}h - {input_end}h")
        print(f"Dự báo: {input_end}h - {input_end + 4}h")

        df_result = test_forecast_interval(
            model, scaler, current_date, input_start, input_end
        )

        if df_result is not None:
            results.append(df_result)

    return results


# Test với 3 khoảng dự báo liên tiếp
test_date = datetime.datetime(2025, 1, 18)
forecast_results = test_multiple_intervals(model, scaler, test_date, num_intervals=3)

# Vẽ biểu đồ kết quả
if forecast_results:
    plt.figure(figsize=(15, 6))
    for i, df in enumerate(forecast_results):
        plt.plot(
            df.index,
            df["Predicted_Power"],
            label=f"Predicted (Interval {i + 1})",
            linestyle="--",
        )
        plt.plot(df.index, df["True_Power"], label=f"Actual (Interval {i + 1})")

    plt.title("Forecast Results for Multiple Intervals")
    plt.xlabel("Time")
    plt.ylabel("Power")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
if df_result is not None:
    print("Kết quả dự báo:")
    print(df_result)

    # Vẽ biểu đồ so sánh dự báo và giá trị thực (nếu có)
    plt.figure(figsize=(12, 6))
    plt.plot(
        df_result.index,
        df_result["Predicted_Power"],
        label="Predicted Power",
        marker="o",
    )
    if not df_result["True_Power"].isnull().all():
        plt.plot(
            df_result.index, df_result["True_Power"], label="True Power", marker="x"
        )
    plt.xlabel("Thời gian")
    plt.ylabel("Công suất")
    plt.title("Dự báo công suất từ 14h đến 18h ngày 18/01/2025")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Không có kết quả dự báo hợp lệ.")

In [ ]:
# tính mape
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
# %% Cell 6: Test model
def test_model(model, scaler, test_start_date, test_end_date):
    global START_DATE, END_DATE
    START_DATE, END_DATE = test_start_date, test_end_date

    date_dirs = [d.strftime("%Y%m%d") for d in pd.date_range(START_DATE, END_DATE)]
    test_image_records_dict = {"b03_": [], "b07_": [], "b13_": [], "b08_": []}
    for date_dir in date_dirs:
        date_path = os.path.join(PATH, date_dir)
        if os.path.exists(date_path):
            for time_dir in os.listdir(date_path):
                time_path = os.path.join(date_path, time_dir)
                if os.path.isdir(time_path):
                    try:
                        dt = datetime.datetime.strptime(
                            f"{date_dir}T{time_dir}", "%Y%m%dT%H%M"
                        )
                        for file in os.listdir(time_path):
                            for prefix in test_image_records_dict.keys():
                                if file.lower().startswith(
                                    prefix
                                ) and file.lower().endswith(
                                    (".jpg", ".jpeg", ".png", ".gif", ".bmp")
                                ):
                                    img_path = os.path.join(time_path, file)
                                    test_image_records_dict[prefix].append(
                                        (img_path, dt)
                                    )
                    except:
                        continue
    for prefix in test_image_records_dict:
        test_image_records_dict[prefix].sort(key=lambda x: x[1])

    sequences_images_test, sequences_times_test = preprocess_images(
        test_image_records_dict
    )
    test_dataset, sequences_times = prepare_dataset_with_processed_images(
        sequences_images_test, sequences_times_test
    )

    filtered_indices = [i for i, t in enumerate(sequences_times) if 5 <= t.hour <= 19]
    if not filtered_indices:
        print("⚠️ Không có dữ liệu trong khoảng 5h-19h!")
        return None, None

    test_loss, test_mae = model.evaluate(test_dataset, verbose=1)
    print(f"✅ Test Loss: {test_loss}")
    print(f"✅ Test MAE: {test_mae}")

    predictions = model.predict(test_dataset)
    true_values = np.concatenate([y.numpy() for _, y in test_dataset], axis=0)
    true_values = scaler.inverse_transform(true_values)
    predictions = scaler.inverse_transform(predictions)

    filtered_true_values = true_values[filtered_indices]
    filtered_predictions = predictions[filtered_indices]
    filtered_times = [sequences_times[i] for i in filtered_indices]

    plt.figure(figsize=(15, 6))
    plt.plot(
        filtered_times,
        filtered_true_values[:, 0],
        label="Giá trị thực tế bước 1",
        color="blue",
    )
    plt.plot(
        filtered_times,
        filtered_predictions[:, 0],
        label="Giá trị dự đoán bước 1",
        color="blue",
        linestyle="--",
    )
    plt.plot(
        filtered_times,
        filtered_true_values[:, 5],
        label="Giá trị thực tế bước 6",
        color="red",
    )
    plt.plot(
        filtered_times,
        filtered_predictions[:, 5],
        label="Giá trị dự đoán bước 6",
        color="red",
        linestyle="--",
    )
    plt.xlabel("Thời gian")
    plt.ylabel("Công suất")
    plt.title("So sánh giá trị thực tế và dự đoán (Bước 1 và Bước 6, 5h-19h)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    def calculate_mape(true, pred):
        mask = true != 0
        return (
            np.mean(np.abs((true[mask] - pred[mask]) / true[mask])) * 100
            if mask.any()
            else float("nan")
        )

    mape_step1 = calculate_mape(filtered_true_values[:, 0], filtered_predictions[:, 0])
    mape_step6 = calculate_mape(filtered_true_values[:, 5], filtered_predictions[:, 5])

    print(f"✅ Sai số phần trăm (MAPE) bước 1: {mape_step1:.2f}%")
    print(f"✅ Sai số phần trăm (MAPE) bước 6: {mape_step6:.2f}%")

    return true_values, predictions


test_start_date = datetime.datetime(2025, 1, 15)
test_end_date = datetime.datetime(2025, 1, 15)
true_values, predictions = test_model(model, scaler, test_start_date, test_end_date)

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(true_values[:, 23], label="Giá trị thực tế bước 6", color="red")
plt.plot(
    predictions[:, 23], label="Giá trị dự đoán bước 6", color="red", linestyle="--"
)
# plt.plot(filtered_times, filtered_true_values[:, 5], label='Giá trị thực tế bước 6', color='red')
# plt.plot(filtered_times, filtered_predictions[:, 5], label='Giá trị dự đoán bước 6', color='red', linestyle='--')
plt.xlabel("Thời gian")
plt.ylabel("Công suất")
plt.title("So sánh giá trị thực tế và dự đoán (Bước 6)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
true_values